In [2]:
import nnts
import nnts.data
import nnts.plotting
import nnts.torch.preprocessing
import nnts.torch.models
import nnts.metrics
import nnts.torch.datasets
import nnts.loggers
import nnts.datasets
from nnts import utils
import nnts.torch.utils
import nnts.torch.trainers
import nnts.metrics
import nnts.torch

import torch
torch.set_printoptions(precision=8, sci_mode=False)
%load_ext autoreload
%autoreload 2

### Load the dataset 
You can easily load any of the Monash datasets using the `load_dataset` function.

In [3]:
df, metadata = nnts.datasets.load_dataset("tourism_monthly")

### Set the Hyperparamters 
You will need to provide an optimizer and a loss function. Other hyperparameters are optional.

In [4]:
params = utils.Hyperparams(
    optimizer=torch.optim.Adam,
    loss_fn=nnts.torch.models.deepar.distr_nll,
    batch_size=32,
    batches_per_epoch=50,
    training_method=utils.TrainingMethod.TEACHER_FORCING,
    model_file_path="logs"
)

### Create covariate features
When using a DeepAR model we recommend using lag features as covariates.

In [5]:
import nnts.lags
lag_seq = nnts.lags.get_lags_for_frequency(metadata.freq)
lag_seq = [lag - 1 for lag in lag_seq if lag > 1]
lag_processor = nnts.torch.preprocessing.LagProcessor(lag_seq)

### Data Processing
We specify how we want to split the data, what transformations we want to apply and how we want to batch and sample the data during training.  

In [5]:
nnts.torch.utils.seed_everything(43)
context_length = metadata.context_length + max(lag_seq)
dataset_options = {
    "context_length": metadata.context_length,
    "prediction_length": metadata.prediction_length,
    "conts": [],
    "lag_seq": lag_seq,
}

trn_dl, test_dl = nnts.torch.utils.create_dataloaders(
    df,
    nnts.datasets.split_test_train_last_horizon,
    context_length,
    metadata.prediction_length,
    Dataset=nnts.torch.datasets.TimeseriesDataset,
    dataset_options=dataset_options,
    Sampler=nnts.torch.datasets.TimeSeriesSampler,
)

### Create the model
Create a Pytorch model.

In [6]:

net = nnts.torch.models.DistrDeepAR(
    nnts.torch.models.deepar.StudentTHead,
    params,
    nnts.torch.preprocessing.masked_mean_abs_scaling,
    1,
    lag_processor=lag_processor,
    scaled_features=[],
    context_length=metadata.context_length,
    cat_idx=None,
    seq_cat_idx=None,
)


### Train the model

In [7]:

trner = nnts.torch.trainers.TorchEpochTrainer(
    net,
    params,
    metadata
)
evaluator = trner.train(trn_dl)

DistrDeepAR(
  (decoder): UnrolledLSTMDecoder(
    (rnn): LSTM(16, 40, num_layers=2, batch_first=True, dropout=0.1)
  )
  (distribution): StudentTHead(
    (main): ModuleList(
      (0-2): 3 x Linear(in_features=40, out_features=1, bias=True)
    )
  )
)
saving model
Epoch 1 Train Loss: 9.228141784667969
saving model
Epoch 2 Train Loss: 8.781664848327637
saving model
Epoch 3 Train Loss: 8.496898651123047
saving model
Epoch 4 Train Loss: 8.413810729980469
saving model
Epoch 5 Train Loss: 8.355382919311523
saving model
Epoch 6 Train Loss: 8.250452041625977
saving model
Epoch 7 Train Loss: 8.136412620544434
saving model
Epoch 8 Train Loss: 8.072505950927734
saving model
Epoch 9 Train Loss: 7.952291965484619
saving model
Epoch 10 Train Loss: 7.919492721557617
saving model
Epoch 11 Train Loss: 7.8912200927734375
saving model
Epoch 12 Train Loss: 7.842233180999756
saving model
Epoch 13 Train Loss: 7.819083213806152
Epoch 14 Train Loss: 7.83964729309082
saving model
Epoch 15 Train Loss: 7.797

### Evaluate

In [8]:
y_hat, y = evaluator.evaluate(
    test_dl, metadata.prediction_length, metadata.context_length
)

test_metrics = nnts.metrics.calc_metrics(
    y_hat, y, nnts.metrics.calculate_seasonal_error(trn_dl, metadata.seasonality)
)
test_metrics

{'mse': 66948348.0,
 'abs_error': 16814716.0,
 'abs_target_sum': 166958480.0,
 'abs_target_mean': 19007.11328125,
 'mase': 1.4237722158432007,
 'mape': 0.21586866676807404,
 'smape': 0.18364249169826508,
 'msmape': 0.18362514674663544,
 'nd': 0.17543545365333557,
 'mae': 1914.243896484375,
 'rmse': 2402.15087890625,
 'seasonal_error': 1543.2191162109375}

In [56]:
nnts.plotting.plotly_forecasts_vs_actuals(y, y_hat)

### Forecast

In [28]:
net = evaluator.net

forecaster = nnts.torch.trainers.TorchForecaster(net)

In [29]:
forecast = forecaster.forecast(test_dl, metadata.prediction_length, metadata.context_length)

In [30]:
forecast

tensor([[[  6641.45605469],
         [  4133.40673828],
         [  3067.53247070],
         ...,
         [  3548.06396484],
         [  4189.00341797],
         [  6918.93017578]],

        [[194433.23437500],
         [159917.76562500],
         [152778.92187500],
         ...,
         [134823.09375000],
         [146866.96875000],
         [201028.07812500]],

        [[ 95321.10937500],
         [ 86995.16406250],
         [103553.07812500],
         ...,
         [133744.62500000],
         [125787.48437500],
         [103492.85937500]],

        ...,

        [[  7711.12939453],
         [ 14208.24902344],
         [ 10858.39062500],
         ...,
         [  7101.75195312],
         [  6812.04296875],
         [  8448.37890625]],

        [[  7796.66650391],
         [  7452.81250000],
         [ 11486.20019531],
         ...,
         [ 14187.00683594],
         [ 10841.85546875],
         [ 10177.20507812]],

        [[  2008.92102051],
         [  1647.43176270],
         [